In [1]:
import torch 
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn 
from PIL import Image, ImageFile
from torchsummary import summary
import warnings
from torchmetrics.classification import Accuracy, Precision, Recall, ConfusionMatrix
from tqdm import tqdm

In [2]:
# ignoring user warnings because it just points out the 'P' images
warnings.simplefilter('ignore', UserWarning)

In [3]:
device = torch.device('mps')
if not(torch.backends.mps.is_available()): 
    device = torch.device('cpu')
device

device(type='mps')

In [4]:
eb3_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(), 
        transforms.Normalize(
            mean=[0.485,0.456, 0.406], 
            std = [0.229, 0.224, 0.225]
        )
    ])


In [5]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
class CustomDataset(ImageFolder): 
    def __init__(self, root, transform=None):
        super().__init__(root, transform=None)
        self.transform = transform
    
    def __getitem__(self, idx):
        
        path, label = self.samples[idx]
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', Image.DecompressionBombWarning)

            try:
                image = Image.open(path).convert('RGB')
                
            except Image.DecompressionBombWarning:
                print(f'Decompression Bomb Warning at {path}')
                raise IndexError(f'Skipping {path} because decompression warning')

            except Exception as e:
                print(e)
                print('PATH = ', path)    
                
        if self.transform: 
            image = self.transform(image)
            
        # label manipulation 
        label = torch.tensor(label)
        label = nn.functional.one_hot(label, num_classes=2)
            
        return image, label
        

In [6]:
BATCH_SIZE = 32
train_link = "datasets/30 k datapoints/train"

dataset = CustomDataset(root = train_link, transform=eb3_transforms)

In [7]:
TRAIN_SPLIT = int(0.8 * len(dataset)) 
train, val = random_split(dataset= dataset, lengths=[TRAIN_SPLIT, len(dataset) - TRAIN_SPLIT])

In [8]:
TrainLoader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
ValLoader = DataLoader(val, batch_size=BATCH_SIZE, shuffle=False)

In [8]:
# printing shapes and testing 
for images, labels in TrainLoader: 
    print(images.shape, labels)
    break

for image, label in ValLoader: 
    print(image.shape, label)
    break

torch.Size([32, 3, 224, 224]) tensor([[1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0]])
torch.Size([32, 3, 224, 224]) tensor([[1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1,

In [6]:
class EB3(nn.Module): 
    def __init__(self, fine_tune = False):
        super(EB3, self).__init__()
        
        # load pretrained model 
        self.eb3 = models.efficientnet_b3(weights = 'DEFAULT')
        
        # freeze the vgg 16 
        if not(fine_tune): 
            for params in self.eb3.parameters(): 
                params.requires_grad = False
        
        self.eb3.classifier = nn.Sequential(
                nn.Linear(self.eb3.classifier[1].in_features, 2048),
                nn.BatchNorm1d(num_features=2048), 
                nn.Dropout(p=0.3),
                nn.ReLU(), 
                nn.Linear(2048, 1024),
                nn.BatchNorm1d(num_features=1024),
                nn.Dropout(p=0.3),
                nn.ReLU(), 
                nn.Linear(1024, 2), 
        )
        
        # making classifier segment trainable 
        for params in self.eb3.classifier.parameters(): 
            params.requires_grad = True
        
    def forward(self, x): 
        return self.eb3(x)
    
    def get_prediction(self, x):
        outputs = self.forward(x)
        outputs = torch.softmax(outputs)
        return torch.argmax(outputs)
        
        
    def StartFineTuning(self, blocks_to_unfreeze=7): 
        ''' unfreeze the last block and train that as well 
        do this only when you train the classifier model 
        '''
        
        for idx in range(blocks_to_unfreeze, len(self.eb3.features)):
            for param in self.eb3.features[idx].parameters():
                param.requires_grad = True
        

In [15]:
model = EB3().to(device=device)

In [16]:
summary(model);

Layer (type:depth-idx)                        Param #
├─EfficientNet: 1-1                           --
|    └─Sequential: 2-1                        --
|    |    └─Conv2dNormActivation: 3-1         (1,160)
|    |    └─Sequential: 3-2                   (3,504)
|    |    └─Sequential: 3-3                   (48,118)
|    |    └─Sequential: 3-4                   (110,912)
|    |    └─Sequential: 3-5                   (638,700)
|    |    └─Sequential: 3-6                   (1,387,760)
|    |    └─Sequential: 3-7                   (4,628,964)
|    |    └─Sequential: 3-8                   (3,284,218)
|    |    └─Conv2dNormActivation: 3-9         (592,896)
|    └─AdaptiveAvgPool2d: 2-2                 --
|    └─Sequential: 2-3                        --
|    |    └─Linear: 3-10                      3,147,776
|    |    └─BatchNorm1d: 3-11                 4,096
|    |    └─Dropout: 3-12                     --
|    |    └─ReLU: 3-13                        --
|    |    └─Linear: 3-14               

In [19]:
def train_eb3(TrainLoader, ValLoader, model, EPOCHS = 5):
    
    # model parameters definition
    lossfn = torch.nn.CrossEntropyLoss()
    LEARNING_RATE = 1E-3
    opt = torch.optim.Adam(
        model.parameters(),
        lr=LEARNING_RATE,
        weight_decay=1e-5
    )
    
    # accuracy 
    accuracy = Accuracy(task='multiclass', num_classes=2).to(device=device)
    
    # training segment
    training_loss_list = []
    val_loss_list = []
    for epoch in range(EPOCHS): 
        running_loss = 0
        TrainLoader_tqdm = tqdm(TrainLoader)
        for image, label in TrainLoader_tqdm:
            
            
            # moving labels and images to GPU
            image = image.to(device=device)
            label = label.to(device=device)
        
            opt.zero_grad()
        
            # predicting and training 
            output = model(image)
            loss = lossfn(output.squeeze(1), label.float())
            loss.backward()
            running_loss += loss.item() / len(TrainLoader)
            
            accuracy.update(output.squeeze(1), label.argmax(dim=1))
            TrainLoader_tqdm.set_postfix({"Training Loss": running_loss})
            opt.step()
            
        train_accuracy = accuracy.compute().item()
        accuracy.reset()
        
        # storing train loss 
        training_loss_list.append(running_loss)
        
        # print progress bar 
        
        
        
        
        
        # validation segment
        val_running_loss = 0
        for image, label in ValLoader: 
            image = image.to(device = device)
            label = label.to(device = device)
            
            # model output
            output = model(image)
            
            # loss computation
            loss = lossfn(output.squeeze(1), label.float())
            val_running_loss += loss.item() / len(ValLoader)
            
            # accuracy computation
            accuracy.update(output.squeeze(1), label.argmax(dim=1))
        
        # validation loss storing 
        val_loss_list.append(val_running_loss)
    
        # final accuracy calculations 
        val_accuracy = accuracy.compute().item()
        accuracy.reset()
        
        # printing metrics
        print(f'''epoch [{epoch+1}/{EPOCHS}]
        \t training loss: {running_loss},
        \t validation loss: {val_running_loss},
        \t Train Accuracy: {train_accuracy},
        \t Val acc: {val_accuracy},
            ''')
    
    return model


In [20]:
EPOCHS = 5
model = train_eb3(TrainLoader, ValLoader, model, EPOCHS=EPOCHS)


100%|██████████| 1200/1200 [40:11<00:00,  2.01s/it, Training Loss=0.338]


epoch [1/5]
        	 training loss: 0.3383557503173749,
        	 validation loss: 0.3021835667143265,
        	 Train Accuracy: 0.8526822924613953,
        	 Val acc: 0.8710416555404663,
            


100%|██████████| 1200/1200 [39:03<00:00,  1.95s/it, Training Loss=0.228]


epoch [2/5]
        	 training loss: 0.22821002292602016,
        	 validation loss: 0.3003210776795944,
        	 Train Accuracy: 0.9076041579246521,
        	 Val acc: 0.879895806312561,
            


100%|██████████| 1200/1200 [39:09<00:00,  1.96s/it, Training Loss=0.132] 


epoch [3/5]
        	 training loss: 0.13167853456068151,
        	 validation loss: 0.3400500608359773,
        	 Train Accuracy: 0.948437511920929,
        	 Val acc: 0.8810416460037231,
            


100%|██████████| 1200/1200 [39:08<00:00,  1.96s/it, Training Loss=0.0703]


epoch [4/5]
        	 training loss: 0.07034948261736038,
        	 validation loss: 0.44281514861931404,
        	 Train Accuracy: 0.9731770753860474,
        	 Val acc: 0.8809375166893005,
            


100%|██████████| 1200/1200 [38:40<00:00,  1.93s/it, Training Loss=0.0468]


epoch [5/5]
        	 training loss: 0.046760336955242905,
        	 validation loss: 0.5168238785148901,
        	 Train Accuracy: 0.9828645586967468,
        	 Val acc: 0.8785416483879089,
            


In [22]:
torch.save(model, './saved models/EB3_no_finetune.pt')

In [26]:
# fine tuning segment 
model.StartFineTuning(blocks_to_unfreeze=7)
summary(model);

Layer (type:depth-idx)                        Param #
├─EfficientNet: 1-1                           --
|    └─Sequential: 2-1                        --
|    |    └─Conv2dNormActivation: 3-1         (1,160)
|    |    └─Sequential: 3-2                   (3,504)
|    |    └─Sequential: 3-3                   (48,118)
|    |    └─Sequential: 3-4                   (110,912)
|    |    └─Sequential: 3-5                   (638,700)
|    |    └─Sequential: 3-6                   (1,387,760)
|    |    └─Sequential: 3-7                   (4,628,964)
|    |    └─Sequential: 3-8                   3,284,218
|    |    └─Conv2dNormActivation: 3-9         592,896
|    └─AdaptiveAvgPool2d: 2-2                 --
|    └─Sequential: 2-3                        --
|    |    └─Linear: 3-10                      3,147,776
|    |    └─BatchNorm1d: 3-11                 4,096
|    |    └─Dropout: 3-12                     --
|    |    └─ReLU: 3-13                        --
|    |    └─Linear: 3-14                   

In [27]:
model = train_eb3(TrainLoader, ValLoader, model, EPOCHS=EPOCHS)

100%|██████████| 1200/1200 [39:44<00:00,  1.99s/it, Training Loss=0.234]


epoch [1/5]
        	 training loss: 0.23395919434726276,
        	 validation loss: 0.22540919076651353,
        	 Train Accuracy: 0.9095051884651184,
        	 Val acc: 0.9069791436195374,
            


100%|██████████| 1200/1200 [39:44<00:00,  1.99s/it, Training Loss=0.144] 


epoch [2/5]
        	 training loss: 0.14427097119429783,
        	 validation loss: 0.20518042450149845,
        	 Train Accuracy: 0.9459114670753479,
        	 Val acc: 0.9205208420753479,
            


100%|██████████| 1200/1200 [39:32<00:00,  1.98s/it, Training Loss=0.107] 


epoch [3/5]
        	 training loss: 0.1070098492623463,
        	 validation loss: 0.2034753813827409,
        	 Train Accuracy: 0.960364580154419,
        	 Val acc: 0.9311458468437195,
            


100%|██████████| 1200/1200 [39:42<00:00,  1.99s/it, Training Loss=0.0774]


epoch [4/5]
        	 training loss: 0.077424864441297,
        	 validation loss: 0.21933525675286858,
        	 Train Accuracy: 0.9712499976158142,
        	 Val acc: 0.9248958230018616,
            


100%|██████████| 1200/1200 [39:39<00:00,  1.98s/it, Training Loss=0.0589]


epoch [5/5]
        	 training loss: 0.05886936199664588,
        	 validation loss: 0.21604542025908194,
        	 Train Accuracy: 0.9786978960037231,
        	 Val acc: 0.9336458444595337,
            


In [7]:
torch.save(model, './saved models/eb3_finetuned.pt')

NameError: name 'model' is not defined

# Testing phase

In [30]:
testlink = "datasets/30 k datapoints/test"
testset = CustomDataset(root= testlink, transform=eb3_transforms)
TestLoader = DataLoader(testset, shuffle=False, batch_size=BATCH_SIZE)

In [36]:

running_loss = 0

# for metrics
accuracy = Accuracy(task='multiclass', num_classes=2).to(device=device)
precision = Precision(task='multiclass', num_classes=2).to(device=device)
recall = Recall(task='multiclass', num_classes=2).to(device=device)
ConMat = ConfusionMatrix(task='multiclass', num_classes=2).to(device=device)

# loss function 
lossfn = torch.nn.CrossEntropyLoss()

# testing phase
TestLoader_tqdm = tqdm(TestLoader)
model.eval()
for image, label in TestLoader_tqdm: 
    image = image.to(device= device)
    label = label.to(device=device)
    
    output = model(image)
    
    loss = lossfn(output, label.float())
    running_loss += loss.item() / len(TestLoader)
    
    TestLoader_tqdm.set_postfix({'Loss':running_loss})
    
    accuracy.update(output, label.argmax(dim=1))
    precision.update(output, label.argmax(dim=1))
    recall.update(output, label.argmax(dim=1))
    ConMat.update(output, label.argmax(dim=1))
    
    

100%|██████████| 375/375 [12:59<00:00,  2.08s/it, Loss=0.207] 


In [46]:
print(f'''Accuracy [{accuracy.compute()}]
    \t Test loss: {running_loss},
    \t Precision: {precision.compute()},
    \t Recall: {recall.compute()},
    \t 
        ''')


Accuracy [0.9354166388511658]
    	 Test loss: 0.20656023084372266,
    	 Precision: 0.9354166388511658,
    	 Recall: 0.9354166388511658,
    	 
        


In [49]:
ConMat.compute()

tensor([[5574,  426],
        [ 349, 5651]], device='mps:0')

# Random test set

In [27]:
def show_image(img):
    """
    Opens and displays an image using the Pillow library.
    """

    import matplotlib.pyplot as plt
    plt.imshow(img)
    plt.axis('off')  # optional: turn off axis
    plt.show()

In [18]:
def inverse_normalize(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)
    return tensor * std + mean


In [12]:
model = torch.load('./saved models/EB3_finetuned.pt').to(device=device)

In [13]:
link = './datasets/real life/'
BATCH_SIZE = 32
randomTestSet = CustomDataset(root = link, transform=eb3_transforms)
TestLoader = DataLoader(randomTestSet, batch_size=BATCH_SIZE, shuffle=True)

In [14]:

running_loss = 0

# for metrics
accuracy = Accuracy(task='multiclass', num_classes=2).to(device=device)
precision = Precision(task='multiclass', num_classes=2).to(device=device)
recall = Recall(task='multiclass', num_classes=2).to(device=device)
ConMat = ConfusionMatrix(task='multiclass', num_classes=2).to(device=device)

accuracy.reset()
precision.reset()
recall.reset()
ConMat.reset()
# loss function 
lossfn = torch.nn.CrossEntropyLoss()

# testing phase
TestLoader_tqdm = tqdm(TestLoader)
model.eval()
for idx, (image, label) in enumerate(TestLoader_tqdm):
    image = image.to(device= device)
    label = label.to(device=device)

    output = model(image)
    
    loss = lossfn(output, label.float())
    running_loss += loss.item() / len(TestLoader)

    TestLoader_tqdm.set_postfix({'Loss':running_loss})

    accuracy.update(output, label.argmax(dim=1))
    precision.update(output, label.argmax(dim=1))
    recall.update(output, label.argmax(dim=1))
    ConMat.update(output, label.argmax(dim=1))
    
    
    if idx == 100: 
        break

100%|██████████| 1/1 [00:00<00:00,  1.52it/s, Loss=1.31]


In [15]:
print(f'''Accuracy [{accuracy.compute()}]
    \t Test loss: {running_loss},
    \t Precision: {precision.compute()},
    \t Recall: {recall.compute()},
    \t 
        ''')


Accuracy [0.6666666865348816]
    	 Test loss: 1.3144327402114868,
    	 Precision: 0.6666666865348816,
    	 Recall: 0.6666666865348816,
    	 
        


In [16]:
ConMat.compute()

tensor([[2, 0],
        [1, 0]], device='mps:0')